In [ ]:
!pip install -U --pre tensorflow=="2.2.0"

In [ ]:
!pip install bing-image-downloader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

from bing_image_downloader import downloader

In [ ]:
items = input("List the comma separated items to be downloaded: ").replace(" ","").split(',')
items.sort()
quantity = int(input("Set the limit: "))

List the comma separated items to be downloaded: car , horse , cycle
Set the limit: 10


In [ ]:
items

['car', 'cycle', 'horse']

In [ ]:
for item in items:
  downloader.download(item, limit=quantity,  output_dir='dataset',adult_filter_off=True, force_replace=False, timeout=60,filter='opencv,".jpg"')

[%] File Downloaded !

[%] Downloading Image #4 from https://4.bp.blogspot.com/-wscEaniqcXk/Uq_ALUHaYwI/AAAAAAAAFYI/T2jWeb10g4g/s640/cars+2014+2.jpg
[%] File Downloaded !

[%] Downloading Image #5 from http://2.bp.blogspot.com/-FnCUNlD0Jj8/UafhnSQN1CI/AAAAAAAAAdY/U4mtxWCBeA8/s640/cars-01.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://1.bp.blogspot.com/-8EELC8O6v18/Uux0Dqr43SI/AAAAAAAAFiI/S9meWMJTBSQ/s1600/2014+cars+7.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://www.9to5carwallpapers.com/wp-content/uploads/2013/06/Black-Bugatti-Veyron-1080p-Wallpaper.jpg
[%] File Downloaded !

[%] Downloading Image #8 from https://2.bp.blogspot.com/-J2o9vyRNDR8/U4Aw4QlMVuI/AAAAAAAAAGg/wZHVjslRBcg/s1600/car+images+3.jpg
[%] File Downloaded !

[%] Downloading Image #9 from http://2.bp.blogspot.com/-T_z0AASLq2U/T5_eCaB79iI/AAAAAAAABfs/WGI4t1vJCx0/s1600/cars+for+2012-7.jpg
[%] File Downloaded !

[%] Downloading Image #10 from https://1.bp.blogspot.com/-0Qlq7IbyhcA/Uf1te

In [ ]:
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript,clear_output
from IPython.display import Image as IPyImage

import tensorflow as tf

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


In [ ]:
pwd

'/content'

book
pen
bottle
phone


In [ ]:
# Load the images as numpy array
gt_dir = []
for item in items:
  for k in os.listdir("/content/dataset/"+item):
    os.rename('/content/dataset/'+item+'/'+k,'/content/dataset/'+item+'/'+k.lower().replace('jpeg','jpg').replace('png','jpg'))
  #print(item)

  train_image_dir = '/content/dataset/'+item
  names=os.listdir(train_image_dir)
  train_images_np = []
  for i in range(1,quantity+1):
    image_path = os.path.join(train_image_dir, 'image_' + str(i) + '.jpg')
    train_images_np.append(load_image_into_numpy_array(image_path))
  gt_dir.append(train_images_np)



        
        Annotaion
          // stuff for image 1
          [
            // stuff for rect 1
            {x, y, w, h},
            // stuff for rect 2
            {x, y, w, h},
            ...
          ]

In [ ]:
gt_itm

[['cat', 0], ['dog', 0], ['elephant', 0]]

In [ ]:
gt_itm =[]
gt_dir_bb = []
for item in items:
  gt_itm.append([item,item.index(item)])
  gt_boxes = []
  colab_utils.annotate(gt_dir[items.index(item)], box_storage_pointer=gt_boxes)
  gt_dir_bb.append(gt_boxes)
  input("Done?")
  clear_output()
  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'--boxes array populated--'

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'--boxes array populated--'

<IPython.core.display.Javascript object>

In [ ]:
gt_dir_bb

[[None, None, None, None, None],
 [None, None, None, None, None],
 [None, None, None, None, None]]

In [ ]:
#Connect the code to Google drive to dump the annotations there
# Now point to the folder with the images to be annotated
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import os
# The path below should point to the directory containing this notebook and the associated utility files
# Change it if necessary
os.chdir('/content/drive/MyDrive/Annotation/')
!ls

In [ ]:
# Finally write the annotations to a file
import json
import numpy as np
data={}
data['bounding box']=[]
data['bounding box'].append(gt_boxes)

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

dumped = json.dumps(data, cls=NumpyEncoder)

In [ ]:
with open('data_others.txt', 'w') as f:
    json.dump(dumped, f)

In [ ]:
gt_boxes[1]=[[0.11774305, 0.34166667, 0.89440972, 0.66770833],[0.11774305, 0.34166667, 0.78940972, 0.56770833]]

In [ ]:
for j in range(quantity):
    print(j, 'index')
    for r in range(len(gt_boxes[j])):
      print(gt_boxes[j][r])
      #print(len(gt_boxes[j]))
      #for r in range(len(gt_boxes[j])):
           # elems = row[r]
            #print(elems)

0 index
[0.12940972 0.0032     1.         0.9632    ]
1 index
[0.11774305, 0.34166667, 0.89440972, 0.66770833]
[0.11774305, 0.34166667, 0.78940972, 0.56770833]
2 index
[0.05774305 0.25362319 0.81274305 0.64492754]


In [ ]:
len(gt_boxes[3])

3

In [ ]:
"""
This script is to convert the txt annotation files to appropriate format needed by YOLO 
"""

#import os
from os import walk, getcwd
from PIL import Image

classes = items
  
"""-------------------------------------------------------------------""" 

""" Configure Paths"""   
for clss in classes:
    #mypath = "Labels/laptop/"   # mypath is the input path of our image txt file folder.firstly we are working on 1st object
    mypath = "/content/dataset/"+clss+"/"
    #outpath = "Labels/output_laptop/"  # where to store the final output txt file
    outpath = "/content/dataset/"+clss+"/" # where to store the final output txt file

    #cls = "laptop"   # on this object we want to train. Remember always train one class at a time. So I have trained first on it and now I am commenting this to work on other class
    cls = clss   # on this object we want to train
    if cls not in classes:
        exit(0)
    cls_id = classes.index(cls)

    wd = getcwd()
    list_file = open('%s/%s_list.txt'%(wd, cls), 'w')

    """ Get input text file list """
    txt_name_list = []
    for (dirpath, dirnames, filenames) in walk(mypath):
        txt_name_list.extend(filenames)
        break
    print(txt_name_list)

    """ Process """
    for j in range(quantity):
        
        txt_name =  'image_'+str(j+1)+".txt"
        """ Open input text files """
        txt_path = mypath + txt_name
        print("Input:" + txt_path)
        lins = []
        #txt_file = open(txt_path, "r")
        
        #lines = txt_file.read().split('\n')   #for ubuntu, use "\r\n" instead of "\n"
        
        """ Open output text files """
        txt_outpath = outpath + 'image_'+str(j+1)+".txt"
        print("Output:" + txt_outpath)
        txt_outfile = open(txt_outpath, "w")
        
        
        """ Convert the data to YOLO format """
        ct = 0
        for r in range(len(gt_boxes[j])):
            elems =  gt_boxes[j][r]

            x = elems[0]
            w = elems[2]
            y = elems[1]
            h = elems[3]
                #
            #img_path = str('%s/dataset/%s/%s.jpg'%(wd, cls,'image_'+str(j+1)+'.jpg'))

            bb = (x,y,w,h)
            print(bb)
            txt_outfile.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

        """ Save those images with bb into list"""
        if(ct != 0):
            list_file.write('%s/dataset/%s/%s.jpg\n'%(wd, cls, os.path.splitext(txt_name)[0]))
                    
    list_file.close()  

['image_3.jpg', 'image_1.jpg', 'image_2.jpg']
Input:/content/dataset/book/image_1.txt
Output:/content/dataset/book/image_1.txt
(0.12940972010294596, 0.0032, 1.0, 0.9632)
Input:/content/dataset/book/image_2.txt
Output:/content/dataset/book/image_2.txt
(0.11774305, 0.34166667, 0.89440972, 0.66770833)
(0.11774305, 0.34166667, 0.78940972, 0.56770833)
Input:/content/dataset/book/image_3.txt
Output:/content/dataset/book/image_3.txt
(0.0577430534362793, 0.2536231884057971, 0.8127430534362793, 0.644927536231884)
['image_3.gif', 'image_1.jpg', 'image_2.jpg']
Input:/content/dataset/pen/image_1.txt
Output:/content/dataset/pen/image_1.txt
(0.12940972010294596, 0.0032, 1.0, 0.9632)
Input:/content/dataset/pen/image_2.txt
Output:/content/dataset/pen/image_2.txt
(0.11774305, 0.34166667, 0.89440972, 0.66770833)
(0.11774305, 0.34166667, 0.78940972, 0.56770833)
Input:/content/dataset/pen/image_3.txt
Output:/content/dataset/pen/image_3.txt
(0.0577430534362793, 0.2536231884057971, 0.8127430534362793, 0.644

In [ ]:
gt_boxes

[array([[0.12940972, 0.0032    , 1.        , 0.9632    ]]),
 [[0.11774305, 0.34166667, 0.89440972, 0.66770833],
  [0.11774305, 0.34166667, 0.78940972, 0.56770833]],
 array([[0.05774305, 0.25362319, 0.81274305, 0.64492754]])]

In [ ]:
#Zip

In [ ]:
!zip -r /content/dataset.zip /content/dataset

In [ ]:
fname = "_"
fname = fname.join(items).replace(" ","")+'.zip'

In [ ]:
fname

'mobile_bottle_keyboard.zip'

In [ ]:
os.rename('dataset.zip',fname)

In [ ]:
#process.py

import glob
import os
import numpy as np
import sys

current_dir = "multiple_images"
split_pct = 10;
file_train = open("train.txt", "w")  
file_val = open("test.txt", "w")  
counter = 1  
index_test = round(100 / split_pct)  
for pathAndFilename in glob.iglob(os.path.join(current_dir, "*.jpg")):  
        title, ext = os.path.splitext(os.path.basename(pathAndFilename))
        if counter == index_test:
                counter = 1
                file_val.write(current_dir + "/" + title + '.jpg' + "\n")
        else:
                file_train.write(current_dir + "/" + title + '.jpg' + "\n")
                counter = counter + 1
file_train.close()
file_val.close()

In [ ]:
ct

1

In [ ]:
print(len(line))

590340


In [ ]:
txt_path

'/content/dataset/bottle/image_2.jpg'